In [2]:
import requests
import xmltodict
import datetime
import pandas as pd
import glob

In [8]:
# 현재 날짜 확인
now = datetime.datetime.now()
nowDate = now.strftime('%Y%m%d')

nowDate

'20210320'

In [12]:
# 현재 디렉토리에 있는 가장 최신 파일 날짜 확인
ls = glob.glob("*.csv")
if not ls:
    oldDate = '20200101'
else:
    ls.sort()
    oldDate = ls[-1][8:-4]

oldDate

In [78]:
# 현재 날짜부터 디렉토리에 있는 가장 최신 파일 날짜까지 데이터 다운로드
url_base = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
url_serviceKey = 'STGRXmZjvEz0TTRJx0Asz574J9AAoG49HNYbsEwJ0W4qsALgk2PK0dMD32ls46%2FtgrAeT6YtsK3QnlvbWJDk0g%3D%3D'
url_pages = '1000'
url_start_date = ls[-1][8:-4]
url_end_date = nowDate
url = url_base + '?serviceKey=' + url_serviceKey + '&pageNo=1&numOfRows=' + url_pages + '&startCreateDt=' + url_start_date + '&endCreateDt=' + url_end_date

In [79]:
req = requests.get(url).content
xmlObject = xmltodict.parse(req)
dict_data = xmlObject['response']['body']['items']['item']

In [81]:
df_new = pd.DataFrame(dict_data)
df_new = df_new.sort_values(by=['stateDt'])
df_new = df_new.drop(df_new.index[0:int(nowDate) - int(ls[-1][8:-4])])

df_new
#   속성 설명
#   DECIDE_CNT          확진자수
#   CLEAR_CNT           격리해제 수
#   EXAM_CNT            검사진행 수
#   DEATH_CNT           사망자 수
#   CARE_CNT            치료중 환자 수
#   RESULT_NEG_CNT      결과 음성 수
#   ACC_EXAM_CNT        누적 검사 수
#   ACC_EXAM_COMP_CNT   누적 검사 완료 수
#   ACC_DEF_RATE        누적 확진률
#   CREATE_DT           등록일시분초
#   UPDATE_DT           수정일시분

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt
0,1.3610022932,7308950,7215932,6567,89949,2021-03-20 09:31:53.146,1693,98209,93018,7117723,454,20210320,00:00,null


In [82]:
df_old = pd.read_csv(ls[-1])
df = df_old.append(df_new)
df.to_csv('covid19_' + nowDate + '.csv', mode='w')

df

,Unnamed: 0,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt
0,413.0,NaN,NaN,NaN,NaN,1,2020-01-31 17:47:33.33,0,0,1,NaN,1,20200101,18:00,2020-02-03 12:21:56.56
1,412.0,NaN,NaN,NaN,NaN,1,2020-01-31 17:47:33.33,0,0,1,NaN,1,20200201,18:00,2020-01-31 17:47:33.33
2,411.0,NaN,NaN,NaN,NaN,2,2020-02-03 12:22:49.49,2,2,2,NaN,2,20200202,09:00,2020-02-03 12:22:49.49
3,410.0,NaN,NaN,NaN,NaN,0,2020-02-03 14:41:17.17,0,15,0,NaN,3,20200203,09:00,2020-02-03 14:41:17.17
4,409.0,NaN,NaN,NaN,NaN,0,2020-02-03 21:26:59.59,0,0,0,NaN,4,20200204,00:00,2020-02-03 21:26:59.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,3.0,1.367474,7126077.0,7048033.0,6447.0,88255,2021-03-16 09:36:13.502,1678,96380,78044,6951653.0,450,20210316,00:00,NaN
411,2.0,1.366371,7171510.0,7088048.0,6349.0,88814,2021-03-17 09:31:37.789,1686,96849,83462,6991199.0,451,20210317,00:00,NaN
412,1.0,1.364885,7218087.0,7128365.0,6428.0,89178,2021-03-18 09:38:40.962,1688,97294,89722,7031071.0,452,20210318,00:00,NaN
413,0.0,1.362163,7264941.0,7176600.0,6544.0,89523,2021-03-19 09:32:03.884,1690,97757,88341,7078843.0,453,20210319,00:00,NaN
